In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
file = "Data\\fivepoints.csv"
df = pd.read_csv(file, sep=';', low_memory=False, encoding='ISO-8859-1')

In [14]:
df.sample(10)

,primary,idll_vo3lv,name,value
542356,542753,2336,timestampcl,"1599730550696,1599730551440"
920697,921094,8440,numbercl,"1,6"
522199,522596,8538,timestampsm,5328
207986,208047,8986,statuscl,"1,1"
195189,195250,8337,numbercl,"1,6,3"
784966,785363,9329,timestampcl,"1601368934523,1601368934867"
930744,931141,9205,patternsm,"0,0,0,1,0,0,0,0"
862581,862978,9986,numbercl,"8,2,3,1"
405148,405545,1323,timestampcl,"1593427248790,1593427249222,1593427249838,1593..."
1116098,1116495,8563,statuscl,"1,1,1"


The dataset is not structured in a way where each row represents a single shape a student has made. In this section, we will analyze the necessary elements to restructure the dataset to the form we will need.

In [16]:
if df['primary'].nunique() == df.shape[0]:
    print("All values in 'primary' are different.")

All values in 'primary' are different.


In [17]:
if df['idll_vo3lv'].nunique() == df.shape[0]:
    print("All values in 'student id' are different.")
else:
    print("no")

no


From this result, we can conclude the column 'primary' represents the primary key and thus they are different in each row, and 'idll_vo3lv' represents the student's id and thus they are not different in each row.

In [18]:
df.dtypes

primary        int64
idll_vo3lv     int64
name          object
value         object
dtype: object

In [20]:
df['name'].unique()

array(['patternsm', 'timestampsm', 'statuscl', 'numbercl', 'timestampcl',
       'timestampload'], dtype=object)

There are 5 unique values in the column 'name'. These values are categories that represent a single shape that a student has made, and thus they will represent the new columns in the restructured dataset. The columns of the restructured dataset will look like this:

In [ ]:
restructured_columns = {
    'student_id': [],
    'patternsm': [],
    'statuscl': [],
    'numbercl': [],
    'timestampcl': [],
    'timestampload': [],
    'timestampsm': [],
    'primary_keys': []
}

In order to improve the speed of my restructuring algorithm, we have sorted the the dataset by the student id in an ascending order, so the test results of each students will be grouped closer to each other, and the algorithm does not have to iterate the whole dataset per loop turn.

In [21]:
sorted_df = df.sort_values(by=['idll_vo3lv'], ascending=True)

The code block below is the algorithm we created to restructure the dataset.

In [22]:
student_ids = df['idll_vo3lv'].unique() 

for studentid in student_ids:
    results = df[df['idll_vo3lv'] == studentid]   
    sorted_results = results.sort_values(by=['primary'], ascending=True)

    # Values
    primary_keys = []
    patternsm = None
    statuscl = None
    numbercl = None
    timestampcl = None
    timestampload = None
    timestampsm = None
    i = 0 

    for _, row in sorted_results.iterrows():

        if row['name'] == 'patternsm':
            patternsm = row['value']
        elif row['name'] == 'statuscl':
            statuscl = row['value']
        elif row['name'] == 'numbercl':
            numbercl = row['value']
        elif row['name'] == 'timestampcl':
            timestampcl = row['value']
        elif row['name'] == 'timestampload':
            timestampload = row['value']
        elif row['name'] == 'timestampsm':
            timestampsm = row['value']
        else: 
            print(f"{row['primary']} has {row['name']}, and it failed")
        
        primary_keys.append(row['primary'])

        # When complete, add the values to the column
        if i == 5:
            new_row = {
                'student_id': studentid,
                'patternsm': patternsm,
                'statuscl': statuscl,
                'numbercl': numbercl,
                'timestampcl': timestampcl,
                'timestampload': timestampload,
                'timestampsm': timestampsm,
                'primary_keys': primary_keys
            }
            for key, value in new_row.items():
                restructured_columns[key].append(value)
            primary_keys = []
            i = 0   
        else:
            i += 1         

restructured_df = pd.DataFrame(restructured_columns)
# restructured_df.to_csv("Data\\restructured_fivepointstest.csv")